In [ ]:
import sympy as sp
from IPython.display import display
sp.init_printing()

In [ ]:
x, y, a, b = sp.symbols('x y a b')

# 1. SymPy: generování kódu

Když máme analytický výsledek ve tvaru symbolického výrazu, nechceme ho ručně přepisovat do numerického kódu.
SymPy umí výraz převést přímo na funkci nebo vygenerovat zdrojový kód v dalších jazycích.

## 1.1 Převod výrazu na Python funkci

Ze symbolického výrazu vytvoříme volatelnou funkci pomocí `lambdify`.

In [ ]:
# první argument je seznam proměnných (podobně jako pro lambda funkce)
f_sympy = (x + sp.pi)**2
display(f_sympy)

f_x = sp.lambdify([x], f_sympy)
print(f_x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xa = np.linspace(-10, 10)
plt.plot(xa, f_x(xa))

Můžeme si zobrazit i zdrojový kód vygenerované funkce pomocí `inspect.getsource`.

In [ ]:
import inspect
print(inspect.getsource(f_x))

`lambdify` umí optimalizovat výpočet přes `cse=True` (common subexpression elimination), tedy vyčlenění opakujících se podvýrazů.
Parametrem `modules` navíc určíme, pro jaký numerický backend má být funkce připravená (např. `numpy`, `jax`).

In [ ]:
slozity_vyraz = (x + sp.pi + sp.exp(x))**2 + sp.sin(x) + sp.sin(x)**2 + sp.exp(x)
display(slozity_vyraz)
f1 = sp.lambdify([x], slozity_vyraz)
print(inspect.getsource(f1))
f2 = sp.lambdify([x], slozity_vyraz, cse=True, modules='numpy')
print(inspect.getsource(f2))

## 1.2 Generování zdrojového kódu v dalších jazycích

Přes modul `codegen` lze výraz exportovat například do C, Fortranu nebo Octave/Matlab.

In [ ]:
import sympy.utilities.codegen as codegen

In [ ]:
# řekněme že chceme někde použít tento výsledek
f = sp.sin( x * y**2) * sp.exp(y)
f

In [ ]:
# exportujeme do jazyka Fortran
f_source = codegen.codegen(("f_fortran", f), "F95", "f_fortran")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka C
f_source = codegen.codegen(("f_C", f), "C", "f_C")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka Octave/Matlab
f_source = codegen.codegen(("f_octave", f), "Octave", "f_octave")
print(f_source[0][1])

## 1.3 Ukázka na komplexnějším příkladu

V této ukázce symbolicky sestavíme model projekce bodu přes sférickou čočku a nakonec z něj vygenerujeme numerickou funkci.

In [ ]:
import sympy as sp
import inspect

# focal length
ff = 0.008

# length of the robot arm + length of the end effector
L = 309.5 * 1e-3

# Define the vectors v and n
v = sp.Matrix([0, 0, -1])
n = sp.Matrix([0, 1, 0])


##########################################
# Camera projection to the plane         #
##########################################

# Define the symbols
alpha, beta, gamma, X, Y, Z, S_x, S_y, S_z = sp.symbols(
    'alpha beta gamma X Y Z S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0],
                [0, sp.cos(alpha), -sp.sin(alpha)],
                [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)],
                [0, 1, 0],
                [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gamma), -sp.sin(gamma), 0],
                [sp.sin(gamma), sp.cos(gamma), 0],
                [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)


# Vector v and calculations for x and o
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
            (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z)

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R * n
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2


# Calculate image plane coordinates f_1 and f_2
f_1 = k_1 * ff / (a + L - ff)
f_2 = k_2 * ff / (a + L - ff)


vysledny_bod = sp.Matrix([f_1, f_2])
vysledny_bod

In [ ]:
# převedeme na Python funkci
kamera_py = sp.lambdify([X, Y, Z, alpha, beta, gamma, S_x, S_y, S_z], vysledny_bod, modules='numpy', cse=True)
print(inspect.getsource(kamera_py))

## 1.4 Další možnosti SymPy

Tímto končí základní přehled. Další témata najdete v oficiálních materiálech:

- [SymPy Tutorial](https://docs.sympy.org/latest/tutorial/index.html)
- [SymPy Documentation](https://docs.sympy.org/latest/index.html)
- [SymPy na GitHubu](https://github.com/sympy/sympy)